In [ ]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import pathlib
import pickle
import itertools
from numba import jit

In [ ]:
import importlib
import evotsc
import evotsc_lib
import evotsc_plot
importlib.reload(evotsc)
importlib.reload(evotsc_lib)
importlib.reload(evotsc_plot)

In [ ]:
exp_path = pathlib.Path('/Users/theotime/Desktop/evotsc/pci/main/')
gen=1_000_000

In [ ]:
rep_dirs = sorted([d for d in exp_path.iterdir() if (d.is_dir() and d.name.startswith("rep"))])
params = evotsc_lib.read_params(rep_dirs[0])
params['m'] = 2.5 # Temporary fix because the parameter wasn't saved

In [ ]:
@jit(nopython=True)
def run_system_numba(nb_genes: int,
                  init_expr: np.ndarray,
                  inter_matrix: np.ndarray,
                  sigma_basal: float,
                  sigma_opt: float,
                  epsilon: float,
                  m: float,
                  sigma_env: float,
                  id_ko:int) -> np.ndarray:

    step_size = 0.5
    stop_dist = 1e-7
    max_eval_steps = 200

    temporal_expr = np.zeros((max_eval_steps+1, nb_genes))

    # Initial values at t = 0
    temporal_expr[0, :] = init_expr
    temporal_expr[0, id_ko] = 0.0


    # Iterate the system
    it = 1
    cont = True
    while cont:
        prev_expr = temporal_expr[it-1, :]
        sigma_local = inter_matrix @ prev_expr
        sigma_total = sigma_basal + sigma_local + sigma_env

        promoter_activity = 1.0 / (1.0 + np.exp((sigma_total - sigma_opt)/epsilon))

        # We subtract 1 to rescale between exp(-m) and 1
        iter_expr = np.exp(m * (promoter_activity - 1.0))

        nouv_expr = step_size * iter_expr + (1 - step_size) * prev_expr
        
        # Knockout
        nouv_expr[id_ko] = 0

        temporal_expr[it, :] = nouv_expr

        # Check if we're done
        dist = np.abs(nouv_expr - prev_expr).sum() / nb_genes

        prev_expr = nouv_expr

        if dist < stop_dist:
            cont = False

        if it == max_eval_steps:
            cont = False
        it += 1

    temporal_expr = temporal_expr[:it, :]

    return temporal_expr

    
def run_system(self, sigma_env, id_ko):

    init_expr = np.array([gene.basal_expression for gene in self.genes])

    self.inter_matrix = self.compute_inter_matrix()

    return run_system_numba(nb_genes=self.nb_genes,
                            init_expr=init_expr,
                            inter_matrix=self.inter_matrix,
                            sigma_basal=self.sigma_basal,
                            sigma_opt=self.sigma_opt,
                            epsilon=self.epsilon,
                            m=self.m,
                            sigma_env=sigma_env,
                            id_ko=id_ko)

In [ ]:
def compute_final_sc_at(self,
                        sigma: float,
                        positions: np.ndarray,
                        id_ko:int) -> np.ndarray:

    gene_positions, genome_size = self.compute_gene_positions(include_coding=True)

    nb_pos = len(positions)
    sc_tsc = np.zeros(nb_pos)

    # Run the individual
    if self.inter_matrix is None:
        self.inter_matrix = self.compute_inter_matrix()
    temporal_expr = run_system(self, sigma, id_ko)
    self.already_evaluated = False # Reset in case the individual is reused
    gene_expr = temporal_expr[-1, :]

    for i_pos, x in enumerate(positions):

        pos_tsc = 0.0

        for i_gene, gene in enumerate(self.genes):

            # We compute the influence of gene i at position x

            # If x is inside gene i, the computation is slightly different
            if ((gene.orientation == 0 and  # Leading
                 x >= gene_positions[i_gene] and
                 x < gene_positions[i_gene] + gene.length) or
                (gene.orientation == 1 and
                 x >= gene_positions[i_gene] - gene.length + 1 and
                 x < gene_positions[i_gene] + 1)):

                if gene.orientation == 0:  # Leading
                    pos_inside = (x - gene_positions[i_gene]) / gene.length
                else:
                    pos_inside = (x - (gene_positions[i_gene] - gene.length + 1)) / gene.length

                # The negative supercoiling is maximal at the promoter
                # (pos_inside = 0), null at half the gene
                # (pos_inside = 1/2), and minimal (ie the positive
                # supercoiling is maximal) at the other end of the gene
                # (pos_inside = 1).

                delta_sc = ((2 * pos_inside - 1) *
                            (1 - gene.length / (2 * self.interaction_dist)) *
                            self.interaction_coef * gene_expr[i_gene])

                if gene.orientation == 1:  # Lagging
                    delta_sc = -delta_sc

                pos_tsc += delta_sc

            else:

                # We use the distance to the middle of the gene to compute
                # the interaction level.
                if gene.orientation == 0:  # Leading
                    pos_i = gene_positions[i_gene] + gene.length // 2
                else:
                    pos_i = gene_positions[i_gene] - gene.length // 2

                pos_i_minus_x = pos_i - x
                pos_x_minus_i = - pos_i_minus_x

                # We want to know whether gene 1/x comes before or after gene 2
                # Before: -------1--2-------- or -2---------------1-
                # After:  -------2--1-------- or -1---------------2-

                if pos_x_minus_i < 0: # -------1--2-------- ou -1---------------2-
                    if pos_i_minus_x < genome_size + pos_x_minus_i: # -------1--2--------
                        distance = pos_i_minus_x
                        x_before_i = True
                    else: # -1---------------2-
                        distance = genome_size + pos_x_minus_i
                        x_before_i = False

                else: # -------2--1-------- ou -2---------------1-
                    if pos_x_minus_i < genome_size + pos_i_minus_x: # -------2--1--------
                        distance = pos_x_minus_i
                        x_before_i = False
                    else:
                        distance = genome_size + pos_i_minus_x
                        x_before_i = True

                # Exit early if genes are too far
                if distance > self.interaction_dist:
                    continue

                if x_before_i:
                    if gene.orientation == 1: # i lagging: +
                        sign_1_on_x = +1
                    else:
                        sign_1_on_x = -1
                else:
                    if gene.orientation == 0: # i leading: +
                        sign_1_on_x = +1
                    else:
                        sign_1_on_x = -1

                # Here, we know that distance <= self.interaction_dist
                strength = 1.0 - distance/self.interaction_dist

                # Supercoiling variations are additive
                delta_sc = sign_1_on_x * strength * self.interaction_coef * gene_expr[i_gene]
                pos_tsc += delta_sc

        sc_tsc[i_pos] = pos_tsc

    return sc_tsc + sigma + self.sigma_basal

In [ ]:
def plot_genome_and_tsc(indiv,
                        sigma,
                        id_ko,
                        show_bar=False,
                        coloring_type='type',
                        use_letters=False,
                        print_ids=False,
                        id_interval=5,
                        plot_name=None):

    # Compute gene positions and activation levels
    gene_pos, genome_length = indiv.compute_gene_positions(include_coding=True)

    # Boolean array, True if a gene's final expression level is > half the max
    if indiv.inter_matrix is None:
        indiv.inter_matrix = indiv.compute_inter_matrix()
    activated_genes = run_system(indiv, sigma, id_ko)[-1, :] > (1 + np.exp(- indiv.m)) / 2

    # Plot
    pos_rect = [0, 0, 1, 1]
    fig = plt.figure(figsize=(9,9), dpi=300)
    ax = fig.add_axes(pos_rect)

    ax.set_xlim(-1.2, 1.2)
    ax.set_ylim(-1.2, 1.2)
    ax.set_aspect('equal')
    circle = plt.Circle(xy=(0, 0), radius=1, linestyle='-', fill=False)
    ax.add_patch(circle)
    ax.set_axis_off()

    text_size = 18

    ## Plot the genes themselves

    if coloring_type == 'type':
        gene_type_color = ['tab:blue', 'tab:red', 'tab:green']
    elif coloring_type == 'on-off':
        colors = plt.cm.get_cmap('tab20').colors
        #                   AB: blue   A:  red    B: green
        gene_type_color = [[colors[1], colors[7], colors[5]],  # light: off
                           [colors[0], colors[6], colors[4]]]  # normal: on
    else:
        gene_colors = mpl.cm.get_cmap('viridis', indiv.nb_genes)(range(indiv.nb_genes))
    gene_types = ['AB', 'A', 'B']

    if use_letters:
        if indiv.nb_genes > 26:
            raise ValueError('Trying to plot with letters on an individual with too many genes')
        letters = 'ABCDEFGHIJKLMNOPQRSTUVWXYZ'

    for i_gene, gene in enumerate(indiv.genes):
        ## Compute the angles of the boundaries of the gene
        start_pos_deg = 360 * gene_pos[i_gene] / genome_length
        if gene.orientation == 0:  # Leading
            end_pos_deg = 360 * (gene_pos[i_gene] + gene.length - 1) / genome_length
        else:
            end_pos_deg = 360 * (gene_pos[i_gene] - (gene.length - 1)) / genome_length
        orient_angle = 360 - (start_pos_deg + end_pos_deg) / 2
        start_pos_rad = np.radians(start_pos_deg)
        end_pos_rad = np.radians(end_pos_deg)
        mid_pos_rad = (start_pos_rad + end_pos_rad) / 2

        ## Plot the gene rectangle
        rect_width = 2 * np.sin((end_pos_rad - start_pos_rad) / 2.0)
        rect_height = 0.1

        x0 = np.sin(start_pos_rad) - 0.5 * rect_height * np.sin(mid_pos_rad)
        y0 = np.cos(start_pos_rad) - 0.5 * rect_height * np.cos(mid_pos_rad)

        if coloring_type == 'type':
            gene_color = gene_type_color[gene.gene_type]
        elif coloring_type == 'on-off':
            if i_gene == id_ko:
                gene_color = 'white'
            else:
                gene_color = gene_type_color[activated_genes[i_gene]][gene.gene_type]
        else:
            if i_gene == id_ko:
                gene_color = 'white'
            else:
                gene_color = gene_colors[i_gene]

        rect = plt.Rectangle(xy=(x0, y0),
                             width=rect_width,
                             height=rect_height,
                             angle=orient_angle, #in degrees anti-clockwise about xy.
                             facecolor=gene_color,
                             edgecolor='black',
                             label=f'Gene {i_gene}')

        ax.add_patch(rect)

        ## Plot the orientation bar and arrow

        # Bar
        x_lin = np.sin(start_pos_rad) + np.array([0.5, 1.0]) * rect_height * np.sin(mid_pos_rad)
        y_lin = np.cos(start_pos_rad) + np.array([0.5, 1.0]) * rect_height * np.cos(mid_pos_rad)

        ax.plot(x_lin, y_lin, color='black', linewidth=1)

        # Arrow
        dx_arr = rect_width * np.cos(mid_pos_rad) / 3.0
        dy_arr = - rect_width * np.sin(mid_pos_rad) / 3.0

        ax.arrow(x_lin[1], y_lin[1], dx_arr, dy_arr, head_width=0.02, color='black')

        ## Print gene ID
        if print_ids and (i_gene % id_interval == 0):
            if use_letters:
                gene_name = letters[i_gene]
            else:
                gene_name = i_gene
            if orient_angle < 120 or orient_angle > 240:  # Top part
                ha = 'left'
                if gene.orientation == 1:  # Lagging
                    ha = 'right'
                ax.text(x=0.915*x0, y=0.915*y0, s=gene_name, rotation=orient_angle,
                        ha=ha, va='bottom', rotation_mode='anchor', fontsize=text_size)
            else:  # Bottom part
                ha = 'right'
                if gene.orientation == 1:  # Lagging
                    ha = 'left'
                ax.text(x=0.93*x0, y=0.93*y0, s=gene_name, rotation=orient_angle+180,
                        ha=ha, va='top', rotation_mode='anchor', fontsize=text_size)


    ## Plot local supercoiling along the genome, at the end of the individual's lifecycle
    sc_ax = fig.add_axes(pos_rect, projection='polar', frameon=False)
    sc_ax.set_ylim(0, 1)

    n = 1000  # the number of data points

    # theta values (see
    # https://matplotlib.org/devdocs/gallery/images_contours_and_fields/pcolormesh_grids.html)
    # To have the crisp version: put n+1 in theta and [data] as the 3rd argument of pcolormesh()
    # To have the blurry version: put n in theta and [data, data] ----------------------------
    theta = np.linspace(0, 2 * np.pi, n)
    radius = np.linspace(.6, .72, 2)

    #data = np.array([theta[:-1]]) #np.array([np.random.random(n) * 2 * np.pi])
    positions = np.linspace(0, genome_length, n, dtype=int)
    data = compute_final_sc_at(indiv, sigma, positions, id_ko) - sigma - indiv.sigma_basal

    min_sc = -0.15
    max_sc = 0.15
    norm = mpl.colors.Normalize(min_sc, max_sc) # Extremum values for the SC level

    if np.min(data) < min_sc or np.max(data) > max_sc:
        print(f'SC values out of bounds! min {np.min(data)}, max {np.max(data)}', file=sys.stderr)

    mesh = sc_ax.pcolormesh(theta, radius, [data, data], shading='gouraud',
                            norm=norm, cmap=plt.get_cmap('seismic'))
    sc_ax.set_yticklabels([])
    sc_ax.set_xticklabels([])
    #sc_ax.spines['polar'].set_visible(False)
    sc_ax.set_theta_zero_location('N')
    sc_ax.set_theta_direction('clockwise')

    # Color bar for the SC level
    if show_bar:
        height = 0.83
        #          [left,  bottom,    width,  height]
        pos_rect = [-0.15, (1 - height)/2, 1, height]
        cbar_ax = fig.add_axes(pos_rect, frameon=False)
        cbar_ax.set_axis_off()

        cbar = fig.colorbar(mesh, ax=cbar_ax, pad=0.0, location='left')
        cbar.set_label('$\sigma_{TSC}$', fontsize=30)
        cbar.ax.invert_yaxis()
        cbar.ax.tick_params(labelsize=text_size)

    ## Legend: gene types and interaction distance

    if coloring_type == 'type':
        draw_legend = True
        patches = ([mpl.patches.Patch(facecolor=color, edgecolor='black', label=label)
            for color, label in zip(gene_type_color, gene_types)])
        ncol = 1

    elif coloring_type == 'on-off':
        draw_legend = True
        patches = ([mpl.patches.Patch(facecolor=color, edgecolor='black', label=label + ' (on)')
                    for color, label in zip(gene_type_color[1], gene_types)] +
                   [mpl.patches.Patch(facecolor=color, edgecolor='black', label=label + ' (off)')
                    for color, label in zip(gene_type_color[0], gene_types)])
        ncol = 2
    else:
        draw_legend = False

    if draw_legend:
        ax.legend(handles=patches, title='Gene type', loc='center', ncol=ncol,
                  handletextpad=0.6, #columnspacing=1.0,
                  title_fontsize=text_size, fontsize=text_size)

    line_len = np.pi*indiv.interaction_dist/genome_length
    if draw_legend:
        line_y = -0.3
    else:
        line_y = -0.1
    ax.plot([-line_len, line_len], [line_y, line_y],
             color='black',
             linewidth=1)
    ax.text(0, line_y - 0.07, 'Gene interaction distance', ha='center', fontsize=text_size)

    ## Wrapping up
    if plot_name:
        plt.savefig(plot_name, dpi=300, bbox_inches='tight')

    plt.show()

    plt.close()

In [ ]:
def plot_base_indiv(rep, gen):
    
    ko_path = exp_path.joinpath(f'kos_rep{rep:02}')
    ko_path.mkdir(exist_ok=True)


    indiv = evotsc_lib.get_best_indiv(exp_path.joinpath(f'rep{rep:02}'), gen)

    evotsc_plot.plot_genome_and_tsc(indiv, sigma=params['sigma_A'], coloring_type='on-off', print_ids=True, 
                                    id_interval=1,
                                    plot_name=ko_path.joinpath(f'ko_genome_and_tsc_gene_0.pdf'))

In [ ]:
def plot_all_kos(rep, rep_dir, gen, sigma):
    
    ko_path = exp_path.joinpath(f'kos_rep{rep:02}')
    ko_path.mkdir(exist_ok=True)
    
    indiv = evotsc_lib.get_best_indiv(rep_dir, gen)
    
    evotsc_plot.plot_genome_and_tsc(indiv, sigma=sigma, coloring_type='on-off', print_ids=True, 
                                    id_interval=1,
                                    plot_name=ko_path.joinpath(f'ko_genome_and_tsc_gene_0.pdf'))
    
    for i_ko in range(indiv.nb_genes):
        plot_genome_and_tsc(indiv, sigma=sigma, id_ko=i_ko, coloring_type='on-off', print_ids=True, 
                            id_interval=1,
                            plot_name=ko_path.joinpath(f'ko_genome_and_tsc_gene_{i_ko:02}.pdf'))

In [ ]:
plot_base_indiv(rep=0, gen=gen)

In [ ]:
for rep, rep_dir in enumerate(rep_dirs[:1]):
    plot_all_kos(rep, rep_dir, gen, sigma=params['sigma_A'])